## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

In [2]:
import numpy as np
import pandas as pd

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [2]:
A = np.random.randint(0, 1000, 10**6)
A

array([761, 922, 814, ..., 287, 580, 735])

In [3]:
def f1(A):
    acc, cnt = 0, 0
    for ai in A:
        bi = ai + 100
        acc += bi
        cnt += 1
    return acc/ cnt

%timeit f1(A)

651 ms ± 75 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
def f2(A):
    acc, cnt = 0, 0
    for ai in A:
        acc +=ai
        cnt += 1
    acc += cnt*100
    return acc/ cnt

In [5]:
%load_ext line_profiler

2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [7]:
df = pd.DataFrame(np.random.randint(0, 1000, size=(2_000_000, 4)),
                  columns=['col1', 'col2', 'col3', 'col4'])
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g']
df['key'] = np.random.choice(letters, 2_000_000, replace=True)

def g(df):
    letters = ['a', 'b', 'c', 'd', 'e']
    dfs = []
    for letter in letters:
        q = df[df['key']==letter]
        dfs.append(q)
    return pd.concat(dfs, axis=0)

%timeit g(A)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

## Лабораторная работа 3

In [4]:
# !pip install line_profiler

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` и без использования срезов таблицы

Б. С использованием метода `DataFrame.iterrows` и с использованием срезов таблицы

В. С использованием метода `DataFrame.mean`

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех вариантов.

In [3]:
reviews= pd.read_csv('data/reviews_sample.csv', sep=',')
reviews

,id,user_id,recipe_id,date,rating,review
0,370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
1,624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
2,187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
3,706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
4,312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...
...,...,...,...,...,...,...
126691,1013457,1270706,335534,2009-05-17,4,This recipe was great! I made it last night. I...
126692,158736,2282344,8701,2012-06-03,0,This recipe is outstanding. I followed the rec...
126693,1059834,689540,222001,2008-04-08,5,"Well, we were not a crowd but it was a fabulou..."
126694,453285,2000242659,354979,2015-06-02,5,I have been a steak eater and dedicated BBQ gr...


In [4]:
recipes = pd.read_csv('data/recipes_sample.csv')
recipes

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,1533,2002-08-29,NaN,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,22724,2002-07-27,NaN,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN
...,...,...,...,...,...,...,...,...
29995,zurie s holey rustic olive and cheddar bread,267661,80,200862,2007-11-25,16.0,this is based on a french recipe but i changed...,10.0
29996,zwetschgenkuchen bavarian plum cake,386977,240,177443,2009-08-24,NaN,"this is a traditional fresh plum cake, thought...",11.0
29997,zwiebelkuchen southwest german onion cake,103312,75,161745,2004-11-03,NaN,this is a traditional late summer early fall s...,NaN
29998,zydeco soup,486161,60,227978,2012-08-29,NaN,this is a delicious soup that i originally fou...,NaN


In [5]:
recipes.dtypes

name               object
id                  int64
minutes             int64
contributor_id      int64
submitted          object
n_steps           float64
description        object
n_ingredients     float64
dtype: object

In [6]:
reviews.dtypes

id            int64
user_id       int64
recipe_id     int64
date         object
rating        int64
review       object
dtype: object

In [7]:
reviews['date'] = pd.to_datetime(reviews['date'], format="%Y-%m-%d")
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126696 entries, 0 to 126695
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   id         126696 non-null  int64         
 1   user_id    126696 non-null  int64         
 2   recipe_id  126696 non-null  int64         
 3   date       126696 non-null  datetime64[ns]
 4   rating     126696 non-null  int64         
 5   review     126679 non-null  object        
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 5.8+ MB


In [10]:
def rating_mean_2010_1():
    s = 0
    k = 0
    for i in reviews.iterrows():
        if i[1].date.year == 2010:
            s += i[1].rating
            k += 1
    return s / k

%time rating_mean_2010_1()

Wall time: 11.7 s


4.4544402182900615

In [18]:
def rating_mean_2010_2():
    s = 0
    k = 0
    for i in reviews[reviews['date'].dt.year == 2010].iterrows():
        s += i[1].rating
        k += 1
    return s / k

%time rating_mean_2010_2()

Wall time: 1.16 s


4.4544402182900615

In [19]:
def rating_mean_2010_3():
    return reviews[reviews['date'].dt.year == 2010]['rating'].mean()

%time rating_mean_2010_3()

Wall time: 32 ms


4.4544402182900615

2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [11]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [21]:
from rating_mean_2010 import rating_mean_2010_1, rating_mean_2010_2, rating_mean_2010_3

In [22]:
# %lprun -f rating_mean_2010_1 rating_mean_2010_1(reviews)

Timer unit: 1e-07 s

Total time: 23.2396 s
File: C:\Users\sergi\PycharmProjects\pythonProject1\seminar3\rating_mean_2010.py
Function: rating_mean_2010_1 at line 4

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     4
     5         1         20.0     20.0      0.0  def rating_mean_2010_1(a: pd.DataFrame):
     6         1         10.0     10.0      0.0      s = 0
     7    126697  191671809.0   1512.8     82.5      k = 0
     8    126696   37462899.0    295.7     16.1      for i in a.iterrows():
     9     12094    3154871.0    260.9      1.4          if i[1].date.year == 2010:
    10     12094     105896.0      8.8      0.0              s += i[1].rating
    11         1         10.0     10.0      0.0              k += 1
    12                                               return s / k

In [23]:
# %lprun -f rating_mean_2010_2 rating_mean_2010_2(reviews)

Timer unit: 1e-07 s

Total time: 2.04959 s
File: C:\Users\sergi\PycharmProjects\pythonProject1\seminar3\rating_mean_2010.py
Function: rating_mean_2010_2 at line 13

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    13
    14         1         19.0     19.0      0.0
    15         1         10.0     10.0      0.0  def rating_mean_2010_2(a: pd.DataFrame):
    16     12095   17081489.0   1412.3     83.3      s = 0
    17     12094    3316376.0    274.2     16.2      k = 0
    18     12094      97932.0      8.1      0.5      for i in a[a['date'].dt.year == 2010].iterrows():
    19         1         29.0     29.0      0.0          s += i[1].rating
    20                                                   k += 1
    21                                               return s / k

In [24]:
# %lprun -f rating_mean_2010_3 rating_mean_2010_3(reviews)

#  rating_mean_2010_3
Timer unit: 1e-07 s

Total time: 0.014963 s

File: C:\Users\sergi\PycharmProjects\pythonProject1\seminar3\rating_mean_2010.py
Function: rating_mean_2010_3 at line 24

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    24                                           def rating_mean_2010_3(a: pd.DataFrame):
    25         1     149630.0 149630.0    100.0      return a[a['date'].dt.year == 2010]['rating'].mean()

3. Вам предлагается воспользоваться функцией, которая собирает информацию, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного прироста в скорости выполнения.

In [12]:
# %%writefile word_reviews_count.py

def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

In [13]:
%%time
result_origin = get_word_reviews_count(reviews)

Wall time: 35 s


In [14]:
# from word_reviews_count import get_word_reviews_count

In [15]:
# %lprun -f get_word_reviews_count get_word_reviews_count(reviews)

Timer unit: 1e-07 s

Total time: 72.8672 s
File: C:\Users\sergi\PycharmProjects\pythonProject1\seminar3\word_reviews_count.py
Function: get_word_reviews_count at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     2                                           def get_word_reviews_count(df):
     3         1         24.0     24.0      0.0      word_reviews = {}
     4    126680  188623473.0   1489.0     25.9      for _, row in df.dropna(subset=['review']).iterrows():
     5    126679   38552533.0    304.3      5.3          recipe_id, review = row['recipe_id'], row['review']
     6    126679    7582991.0     59.9      1.0          words = review.split(' ')
     7   6918689   37681643.0      5.4      5.2          for word in words:
     8   6792010   57679549.0      8.5      7.9              if word not in word_reviews:
     9    174944    1674011.0      9.6      0.2                  word_reviews[word] = []
    10   6792010   61677957.0      9.1      8.5              word_reviews[word].append(recipe_id)
    11
    12         1         50.0     50.0      0.0      word_reviews_count = {}
    13    126680  182551755.0   1441.0     25.1      for _, row in df.dropna(subset=['review']).iterrows():
    14    126679   22078855.0    174.3      3.0          review = row['review']
    15    126679    7601167.0     60.0      1.0          words = review.split(' ')
    16   6918689   38434900.0      5.6      5.3          for word in words:
    17   6792010   84533462.0     12.4     11.6              word_reviews_count[word] = len(word_reviews[word])
    18         1         30.0     30.0      0.0      return word_reviews_count


In [50]:
def get_word_reviews_count_opt(df):
    word_reviews_count = {}
    for row in df.dropna(subset=['review'])['review'].str.split(' '):
        for word in row:
            if word in word_reviews_count:
                word_reviews_count[word] += 1
            else:
                word_reviews_count[word] = 1
    return word_reviews_count

In [53]:
%%time
result_opt = get_word_reviews_count_opt(reviews)

Wall time: 5.76 s


In [54]:
result_origin == result_opt

True

3. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего процентного отклонения значения рейтинга для отзыва от среднего значения рейтинга для этого отзыва.
    1. Без использования массивов `numpy` и `numba`
    2. Без использования массивов `numpy`, но с использованием `numba` - смысла нет, так как numba не работает типами данных pandas!!!
    3. С использованием массивов `numpy`, но без использования `numba`
    4. C использованием массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.

In [8]:
#1
def MAPE_1(df: pd.DataFrame):
    rating_clear = df[df['rating'] != 0]['rating']
    mean = rating_clear.mean()
    return rating_clear.apply(lambda x: abs(mean - x)).sum() / mean * 100 / rating_clear.count()

In [9]:
%time MAPE_1(reviews)

Wall time: 67 ms


11.079150232267242

In [11]:
def MAPE_3(df: pd.DataFrame):
    rating_clear = df[df['rating'] != 0]['rating'].to_numpy()
    mean = rating_clear.mean()
    return np.absolute(rating_clear - mean).sum() / mean * 100 / rating_clear.shape[0]

In [18]:
%timeit MAPE_3(reviews)

11.2 ms ± 137 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [13]:
from numba import njit

@njit
def help_M_4(a: np.array):
    mean = a.mean()
    return np.absolute(a - mean).sum() / mean * 100 / a.shape[0]


def MAPE_4(df: pd.DataFrame):
    rating_clear = df[df['rating'] != 0]['rating'].to_numpy()
    return help_M_4(rating_clear)

In [19]:
%timeit MAPE_4(reviews)

10.9 ms ± 55.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
